In [1]:
import os
import time
import struct
import numpy as np
import pandas as pd

In [2]:
from pathlib import Path
os.chdir(Path(os.getcwd()).parent)
from libs.utils import Utils

In [ ]:
tdx_root = 'C:\\new_gjzq_v6'

In [ ]:
class TDX:
    def __init__(self, tdx_root='C:\\new_gjzq_v6'):
        self.root = tdx_root
        
    def get_tdx_gainian(self):

        fname = os.path.join(self.root,'T0002', 'hq_cache', 'block_gn.dat')
        result = {}
        if type(fname) is not bytearray:
            with open(fname, "rb") as f:
                data = f.read()
        else:
            data = fname

        pos = 384
        (num, ) = struct.unpack("<H", data[pos: pos + 2])
        pos += 2
        for i in range(num):
            blockname_raw = data[pos: pos + 9]
            pos += 9
            name = blockname_raw.decode("gbk", 'ignore').rstrip("\x00")
            stock_count, block_type = struct.unpack("<HH", data[pos: pos + 4])
            pos += 4
            block_stock_begin = pos
            codes = []
            for code_index in range(stock_count):
                one_code = data[pos: pos + 7].decode("utf-8", 'ignore').rstrip("\x00")
                codes.append(one_code)
                pos += 7

            gn = {}
            gn["name"] = name
            gn["block_type"] = block_type
            gn["stock_count"] = stock_count
            gn["codes"] = codes
            result[name] = gn

            pos = block_stock_begin + 2800

        return result
    
    
    def get_tdx_hangye(self):

        file_hangye = os.path.join(self.root, 'incon.dat')
        assert os.path.exists(file_hangye)
        file_stock_hangye = os.path.join(self.root, 'T0002', 'hq_cache','tdxhy.cfg')
        assert os.path.exists(file_stock_hangye), file_stock_hangye

        result = {}
        with open(file_hangye, "rt", encoding='gb2312') as f:
            isTDXHY = False
            for line in f:
                line = line.rstrip()
                if not isTDXHY and line != '#TDXNHY':
                    continue
                elif not isTDXHY and line == '#TDXNHY':
                    isTDXHY = True
                    continue
                elif isTDXHY and line == '######':
                    isTDXHY = False
                    break
                code, name = line.split('|')
                result[code] = {}
                result[code]['code'] = code
                result[code]['name'] = name
                result[code]['codes'] = []

        with open(file_stock_hangye, "rt", encoding='gb2312') as f:
            for line in f:
                line = line.rstrip()
                market_code, stock_code, tdxhy_code, swhy_code, unknown_code = line.split("|")
                stock_code = stock_code.strip()

                if tdxhy_code != 'T00':
                    result[tdxhy_code]['codes'].append(stock_code)
        return result
    
    
    def get_tdx_zhishu(self):

        tdxzs_cfg = os.path.join(self.root, 'T0002', 'hq_cache', 'tdxzs.cfg')
        gainian = self.get_tdx_gainian()
        hangye = self.get_tdx_hangye()

        result = {}
        with open(tdxzs_cfg, "rt", encoding='gb2312') as f:
            for line in f:
                line = line.rstrip()
                zs_name, zs_code, zs_type, num_1, num_2, key = line.split('|')

                if key in gainian:
                    if zs_code in result:
                        print('------------------------------------------------------')
                        print('in result key: ', key, zs_name, zs_code)
                        print('gainian: ', key, gainian[key])
                        continue
                    else:
                        if len(gainian[key]['codes']) == 0:
                            continue
                        zs = {}
                        zs['code'] = zs_code
                        zs['name'] = gainian[key]['name']
                        zs['codes'] = gainian[key]['codes']
                        result[zs_code] = zs

                if key in hangye:
                    if zs_code in result:
                        print('------------------------------------------------------')
                        print('in result key: ', key, zs_name, zs_code)
                        print('hangye: ', key, hangye[key])
                        continue
                    else:
                        if len(hangye[key]['codes']) == 0:
                            continue
                        zs = {}
                        zs['code'] = zs_code
                        zs['name'] = hangye[key]['name']
                        zs['codes'] = hangye[key]['codes']
                        result[zs_code] = zs

        return result
    
    
    def kline(self, symbols):
        kline = {}
        for symbol in symbols:
            market = 'sh' if symbol[0]=='6' else 'sz'
            file = f"{tdx_root}\\vipdoc\\{market}\\lday\\{market}{symbol}.day"
#             print(file)
            if not os.path.exists(file):
                print(f'{file} does not exist.')
                continue
                
            with open(file, 'rb') as f:
                buf = f.read()
                buf_size = len(buf)
                rec_count = int(buf_size / 32)

                data = []
                for i in range(rec_count):
                    a = struct.unpack('IIIIIfII', buf[i*32:(i+1)*32])
    #                 print(a)
                    data.append({
                        'dt':a[0],
                        'open':a[1]/100,
                        'high':a[2]/100,
                        'low':a[3]/100,
                        'close':a[4]/100,
                        'amount':a[5],
                        'volume':a[6]
                    })
            
            df = pd.DataFrame(data)
            df.set_index('dt', inplace=True)
            kline[symbol] = df
            
        return kline
    
    
    def is_local_tdx_data_outdated(self):
        today = int(time.strftime('%Y%m%d'))
        last_trade_date = int(Utils.get_last_trade_date())
        df = self.kline(['399001'])['399001']
        
        if df.index[-1] == last_trade_date:
            return False
        elif today != last_trade_date:
            return True
        elif time.time() > time.mktime(time.strptime(f'{today} 16:00:00', '%Y%m%d %H:%M:%S')):
            return True
        else:
            return False
        
        
        

In [ ]:
tdx = TDX()

In [ ]:
tdx.is_local_tdx_data_outdated()

In [ ]:
gainian = tdx.get_tdx_gainian()
gainian

In [ ]:
hangye = tdx.get_tdx_hangye()
hangye

In [ ]:
zhishu = tdx.get_tdx_zhishu()
zhishu

In [ ]:
assert tdx.is_local_tdx_data_outdated() is not True
symbols = Utils.get_running_symbols()
klines = tdx.kline(symbols)
ma5vpm = []
sum5 = []
sum10 = []
sum20 = []
sum30 = []
sum60 = []
for symbol in symbols:
    kline = klines[symbol]
    ma5vpm.append(kline.iloc[0-min(5,len(kline)):]['volume'].sum()/1200)
    sum5.append(kline.iloc[-4:]['close'].sum() if len(kline) >= 4 else np.nan)
    sum10.append(kline.iloc[-9:]['close'].sum() if len(kline) >= 9 else np.nan)
    sum20.append(kline.iloc[-19:]['close'].sum() if len(kline) >= 19 else np.nan)
    sum30.append(kline.iloc[-29:]['close'].sum() if len(kline) >= 29 else np.nan)
    sum60.append(kline.iloc[-59:]['close'].sum() if len(kline) >= 59 else np.nan)


In [ ]:
assert tdx.is_local_tdx_data_outdated() is not True
symbols = Utils.get_running_symbols()
klines = tdx.kline(symbols)

In [ ]:
t = time.time()
ma5vpm = [ klines[symbol].iloc[0-min(5,len(klines[symbol])):]['volume'].sum()/1200 for symbol in symbols ]
sum5 = [ klines[symbol].iloc[-4:]['close'].sum() if len(klines[symbol]) >= 4 else np.nan for symbol in symbols ]
sum10 = [ klines[symbol].iloc[-9:]['close'].sum() if len(klines[symbol]) >= 9 else np.nan for symbol in symbols ]
sum20 = [ klines[symbol].iloc[-19:]['close'].sum() if len(klines[symbol]) >= 19 else np.nan for symbol in symbols ]
sum30 = [ klines[symbol].iloc[-29:]['close'].sum() if len(klines[symbol]) >= 29 else np.nan for symbol in symbols ]
sum60 = [ klines[symbol].iloc[-59:]['close'].sum() if len(klines[symbol]) >= 59 else np.nan for symbol in symbols ]
time.time() - t

In [ ]:
t = time.time()
ma5vpm = []
sum5 = []
sum10 = []
sum20 = []
sum30 = []
sum60 = []
for symbol in symbols:
    kline = klines[symbol]
    ma5vpm.append(kline.iloc[0-min(5,len(kline)):]['volume'].sum()/1200)
    sum5.append(kline.iloc[-4:]['close'].sum() if len(kline) >= 4 else np.nan)
    sum10.append(kline.iloc[-9:]['close'].sum() if len(kline) >= 9 else np.nan)
    sum20.append(kline.iloc[-19:]['close'].sum() if len(kline) >= 19 else np.nan)
    sum30.append(kline.iloc[-29:]['close'].sum() if len(kline) >= 29 else np.nan)
    sum60.append(kline.iloc[-59:]['close'].sum() if len(kline) >= 59 else np.nan)
time.time() - t

In [ ]:
r = []

In [ ]:
[r.append(_) for _ in range(10)]

In [ ]:
for _, v in enumerate(sum60):
    if v is np.nan:
        print(_, symbols[_])

In [ ]:
symbols.index('688317')

In [ ]:

[ klines[symbol].iloc[-4:]['close'].sum() if len(klines[symbol]) >= 4 else np.nan ]

In [ ]:
klines[symbol]

In [ ]:
time.strftime('%H%M%S', time.localtime(1610673300))

In [ ]:
today = int(time.strftime('%Y%m%d'))


In [ ]:
time.time()

In [ ]:
time.strptime(f'{today} 16:00:00', '%Y%m%d %H:%M:%S')

In [ ]:
symbols = Utils.get_running_symbols()
klines = tdx.kline(symbols)

In [ ]:
klines['002717']

In [3]:
from libs.tdx import TDX
from libs.dailydata import DailyData
from libs.cython.compute import compute_stats

In [4]:
date = time.strftime('%Y%m%d')
date

'20210622'

In [5]:
data = DailyData.load(dt=date)

In [6]:
st = time.time()
for _ in range(len(data.check_points)):
    time_lapse = data.get_time_lapse(_)
    ma5pm_anchor_idx = data.get_ma5pm_anchor_idx(_)
    fs5p = data.snapshots[ma5pm_anchor_idx]
    compute_stats(data.snapshots[_], data.basics, data.statistic[_], fs5p, time_lapse)
et = time.time()
et - st

1.0132043361663818

In [7]:
ss = data.get_snapshot('13:07:45')
ss

,datetime,timestamp,name,open,close,now,high,low,turnover,volume,zhangfu,junjia,liangbi,zhangsu,tingban,ma5
symbol,,,,,,,,,,,,,,,,
000001,13:07:45,1624338465,平安银行,22.38,22.16,22.73,22.77,22.25,"43,408,668.00","979,494,003.79",2.57,22.56,1.40,1.56,0.00,22.78
000002,13:07:45,1624338465,万 科Ａ,24.30,24.20,24.16,24.47,24.16,"34,376,457.00","835,808,896.75",-0.17,24.31,0.96,-0.58,0.00,24.58
000004,13:07:45,1624338465,国华网安,20.30,20.77,20.56,20.56,20.02,"4,443,189.00","90,042,699.95",-1.01,20.27,0.55,1.28,0.00,20.44
000005,13:07:45,1624338465,ST星源,1.82,1.83,1.88,1.89,1.81,"6,709,009.00","12,380,126.08",2.73,1.85,2.02,3.30,0.00,1.84
000006,13:07:45,1624338465,深振业Ａ,5.24,5.24,5.24,5.27,5.22,"3,270,300.00","17,174,296.00",0.00,5.25,1.18,0.00,0.00,5.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688777,13:07:45,1624338465,中控技术,91.80,92.00,91.44,92.81,89.60,"571,489.00","51,843,772.00",-0.61,90.72,0.97,-0.39,0.00,90.11
688788,13:07:45,1624338465,科思科技,139.20,137.93,135.79,139.20,133.79,"584,001.00","79,228,813.00",-1.55,135.67,1.83,-2.45,0.00,133.07
688819,13:07:45,1624338465,天能股份,42.12,42.12,41.90,42.27,41.65,"1,237,633.00","51,811,621.00",-0.52,41.86,0.69,-0.52,0.00,41.07


In [8]:
ss.loc['600354']

datetime          13:07:45
timestamp       1624338465
name                  敦煌种业
open                  5.80
close                 5.76
now                   6.27
high                  6.34
low                   5.72
turnover     42,527,873.00
volume      262,371,856.00
zhangfu               8.85
junjia                6.17
liangbi               2.27
zhangsu               8.10
tingban               0.50
ma5                   5.74
Name: 600354, dtype: object

In [9]:
tdx = TDX()

In [10]:
zhishu = tdx.get_tdx_zhishu()
zhishu

{'880302': {'code': '880302',
  'name': '煤炭开采',
  'codes': ['000552',
   '000780',
   '000937',
   '000968',
   '000983',
   '002128',
   '600121',
   '600123',
   '600188',
   '600348',
   '600395',
   '600397',
   '600403',
   '600508',
   '600546',
   '600758',
   '600971',
   '600985',
   '600997',
   '601001',
   '601088',
   '601101',
   '601225',
   '601666',
   '601699',
   '601898',
   '601918']},
 '880303': {'code': '880303',
  'name': '焦炭加工',
  'codes': ['000723',
   '600408',
   '600725',
   '600740',
   '600792',
   '601011',
   '601015',
   '603113']},
 '880306': {'code': '880306',
  'name': '水力发电',
  'codes': ['000601',
   '000722',
   '000791',
   '000993',
   '002039',
   '600025',
   '600101',
   '600116',
   '600236',
   '600310',
   '600452',
   '600505',
   '600644',
   '600674',
   '600868',
   '600886',
   '600900',
   '600969',
   '600979',
   '600995']},
 '880307': {'code': '880307',
  'name': '火力发电',
  'codes': ['000027',
   '000037',
   '000531',
   '000539',

In [11]:
symbol2zhishu = {}
for zhishu_symbol in zhishu:
    for symbol in zhishu[zhishu_symbol]['codes']:
        if symbol not in symbol2zhishu:
            symbol2zhishu[symbol] = []
        if zhishu_symbol not in symbol2zhishu[symbol]:
            symbol2zhishu[symbol].append(zhishu_symbol)
symbol2zhishu

{'000552': ['880302', '880524'],
 '000780': ['880302', '880516', '880917'],
 '000937': ['880302', '880505', '880911'],
 '000968': ['880302', '880553'],
 '000983': ['880302', '880550'],
 '002128': ['880302', '880544', '880582', '880917', '880964'],
 '600121': ['880302'],
 '600123': ['880302', '880505', '880938'],
 '600188': ['880302', '880501'],
 '600348': ['880302', '880505'],
 '600395': ['880302', '880527', '880564'],
 '600397': ['880302'],
 '600403': ['880302', '880516'],
 '600508': ['880302'],
 '600546': ['880302', '880544', '880595'],
 '600758': ['880302', '880519', '880705', '880909'],
 '600971': ['880302'],
 '600985': ['880302', '880503', '880524'],
 '600997': ['880302', '880705'],
 '601001': ['880302', '880505'],
 '601088': ['880302', '880501', '880515', '880917'],
 '601101': ['880302', '880505'],
 '601225': ['880302'],
 '601666': ['880302', '880505'],
 '601699': ['880302'],
 '601898': ['880302', '880501', '880917'],
 '601918': ['880302', '880917'],
 '000723': ['880303', '880705

In [43]:
zt_indices = []
for _ in range(len(data.check_points)):
    cp = data.check_points[_]
    dt = time.strftime('%H:%M:%S', time.localtime(cp))
    indices = np.argwhere(data.statistic[_,:,4]>0)
    
    new_zt_indices = []
    for idx in indices:
        if idx[0] not in zt_indices:
            new_zt_indices.append(idx[0])
        
    if new_zt_indices:
        zt_indices = zt_indices + new_zt_indices
        print(dt, new_zt_indices)

09:25:15 [184, 279, 311, 313, 911, 973, 1064, 1098, 1205, 1294, 2509, 2588, 2961, 2984, 3171, 3328, 3351, 3691, 3805, 3988]
09:30:00 [2737]
09:30:10 [13, 335, 3894]
09:30:20 [3109]
09:30:25 [2887]
09:30:55 [707]
09:31:15 [2538]
09:31:20 [2730]
09:31:50 [2583]
09:32:10 [2577]
09:33:50 [2621]
09:34:05 [2340]
09:34:20 [1122]
09:34:55 [3049]
09:35:30 [961]
09:37:05 [2673]
09:37:15 [1510]
09:37:30 [1035]
09:37:35 [2721]
09:38:35 [3599]
09:39:30 [3910]
09:39:35 [2486]
09:40:45 [2711]
09:41:15 [2537]
09:42:25 [432]
09:43:30 [2519]
09:43:35 [1065]
09:43:45 [3822]
09:45:15 [120]
09:45:45 [542]
09:46:25 [2712]
09:46:45 [2985]
09:46:50 [1061, 1109]
09:47:15 [5]
09:47:50 [1637, 3075]
09:48:15 [3886]
09:50:45 [3042]
09:51:10 [3114]
09:51:55 [2770]
09:53:00 [805]
09:53:10 [368]
09:54:05 [1323]
09:55:20 [3403]
10:00:30 [3754]
10:00:40 [343]
10:00:50 [340]
10:04:25 [2816]
10:04:50 [1331]
10:06:30 [1967]
10:08:00 [1613]
10:08:35 [2854]
10:09:15 [463]
10:16:45 [3127]
10:19:00 [2525]
10:20:10 [627]
10:22

In [47]:
for idx in zt_indices:
    print(data.symbols[idx], data.names[idx])

000611 *ST天
000752 *ST西
000802 ST北文
000806 *ST银
002464 *ST众
002530 金财互联
002622 融钰集团
002656 ST摩登
002772 众兴菌业
002870 香山股份
600112 *ST天
600203 福日电子
600666 ST瑞德
600695 *ST绿
600905 三峡能源
601567 三星医疗
601665 齐鲁银行
603529 爱玛科技
603729 ST龙韵
605169 洪通燃气
600382 *ST广
000017 *ST中
000835 *ST长
603909 合诚股份
600834 申通地铁
600582 天地科技
002256 *ST兆
600146 *ST环
600375 汉马科技
600198 *ST大
600191 *ST华
600241 ST时万
300921 南凌科技
002681 奋达科技
600767 ST运盛
002516 旷达科技
600306 *ST商
300057 万顺新材
002592 ST八菱
600365 ST通葡
603303 得邦照明
603937 丽岛新材
600084 *ST中
600353 旭光电子
600145 *ST新
000980 *ST众
600122 ST宏图
002623 亚玛顿
603779 ST威龙
000530 冰山冷热
002089 ST新海
600354 敦煌种业
600696 岩石股份
002619 *ST艾
002667 鞍重股份
000007 *ST全
300189 神农科技
600796 钱江生化
603897 长城科技
600758 辽宁能源
600839 四川长虹
600428 中远海特
002356 *ST赫
000893 亚钾国际
002901 大博医疗
601908 京运通
603657 春光科技
000852 石化机械
000848 承德露露
600500 中化国际
002910 庄园牧场
300526 中潜股份
300164 通源石油
600545 卓郎智能
002006 精功科技
600856 ST中天
600129 太极集团
002175 *ST东
000687 *ST华
000525 ST红太
002123 梦网科技
000587 *ST金
600721 *ST百
002520

In [56]:
st_zt_indices = list([idx for idx in zt_indices if 'ST' in data.names[idx]])
st_zt_indices

[184,
 279,
 311,
 313,
 911,
 1098,
 2509,
 2961,
 2984,
 3805,
 2737,
 13,
 335,
 707,
 2538,
 2583,
 2577,
 2621,
 3049,
 2673,
 1035,
 2721,
 2486,
 2537,
 432,
 2519,
 3822,
 542,
 1061,
 5,
 805,
 3127,
 627,
 235,
 116,
 164,
 3007,
 216,
 2678,
 599,
 295,
 426,
 942,
 566,
 2491,
 1080,
 2646,
 3438,
 462]

In [57]:
len(zt_indices)-len(st_zt_indices)

96

'600203'

array([0.00, 0.00, 0.00, ..., 0.00, 0.00, 0.00])